I'm having a look at how to optimise mashed shuffling (for magic reasons!)

In [2]:
%matplotlib inline

import os

import random
import pickle

import json

import numpy as np
from scipy.stats import multivariate_normal as multinomial
from scipy.stats import rv_discrete
from scipy.stats import norm

import pandas as pd

from matplotlib import pyplot
import seaborn

from mtgsdk import Card

A function to return card names from the mtgsdk.

In [3]:
def get_card(card_name):
    print(f'Retrieving: {card_name.title()}.')
    card = Card.where(name=card_name).all()[0]
    print(f'{card_name.title()} retrieved!\n')
    
    return card

A rifle shuffle function.
TODO : make this work for an arbitrary left/right hand drop

In [4]:
def rifle_shuffle(deck_list):
    
    pile = []
    
    a = [a for a in range(28,33)]
    split = random.choices(a)[0]
    
    d = [a for a in range(1,2)]
    
    half1 = deck_list[:split]
    half2 = deck_list[split:]
    
    while (half1 and half2):
        half1_drop_number = random.choices(d)[0]
        half2_drop_number = random.choices(d)[0]
        pile = pile + half1[-half1_drop_number:] + half2[-half2_drop_number:]
        half1 = half1[:-half1_drop_number]
        half2 = half2[:-half2_drop_number]
        
    if half1:
        pile = half1 + pile
        
    if half2:
        pile = half2 + pile
    
    return pile

Overhand shuffling algorithm.

In [5]:
def overhand_shuffle(deck_list):
    a = [x for x in range(100)]
    
    pdf = multinomial.pdf(a, mean=7, cov=6)
    norm_pdf = [(val/sum(pdf)) for val in pdf]
        
    right_hand = deck_list[:]
    left_hand = []
    while len(right_hand) > 7:
        overhand_dist = rv_discrete(a=1, b=len(right_hand), values=(a,norm_pdf))
        drop = overhand_dist.rvs(size=1)[0]
        left_hand = right_hand[:drop] + left_hand
        right_hand = right_hand[drop:]
    left_hand = right_hand + left_hand
    right_hand = []
    
    return left_hand

Function to combine different shuffle types, and examine the hand retrieved from this process.
TODO : put in the 'qualities' stuff.

In [6]:
def shuffle(deck, search_criteria, overhand_interval):
    stats = []

    for i in range(1000):
        deck = list(random.sample(deck_list, len(deck_list)))
        hand = [item for item in deck[:7]]
        shuffles = 0
        overhand_shuffles = False
        
        hand_types = [item.type for item in hand]
        
        hand_cards = [item.name for item in hand]
        
        hand_qualities = {}
        
        for key in search_criteria.keys():
            hand_qualities[key] = get_hand_attribute(hand, key)
        
        while not (search_criteria['quality'].issubset(hand_qualities['quality'])
                   and hand_types.count('Land') == 1):
            deck = rifle_shuffle(deck)
            hand = [item for item in deck[:7]]
            
            hand_qualities['quality'] = get_hand_attribute(hand, 'quality')
            
            hand_types = [item.type for item in hand]
        
            hand_cards = [item.name for item in hand]
            
            
            shuffles = shuffles + 1
            if (shuffles % overhand_interval) == 0:
                overhand_shuffles = True
            else:
                overhand_shuffles = False

            #if shuffles >= 15:
            #    break

        if overhand_shuffles:
            deck = overhand_shuffle(deck)

        stats.append({'shuffles': shuffles, 'hand': hand})

    return stats

In [7]:
def get_hand_attribute(hand, attribute):
    quality_list = [make_list(getattr(card, str(attribute))) for card in hand]
    quality_list = [item for sublist in quality_list for item in sublist]
    
    quality_set = set(quality_list)
    
    return quality_set

In [8]:
def make_list(item):
    if not isinstance(item, list):
        item = [item]
    
    return item

In [9]:
deck_json = 'deck.json'
deck_pickle = 'deck.pkl'

if os.path.isfile(deck_pickle):
    with open(deck_pickle, 'rb') as f:
        deck_list = pickle.load(f)
else:
    with open(deck_json, 'rb') as f:
        card_list = json.load(f)

    cards = {}

    for i in card_list.keys():
        name = card_list[i]['name']
        quantity = card_list[i]['quantity']
        quality = card_list[i]['quality']
        card = get_card(name)
        card.quality = quality
        cards[name] = [card, quantity]

    deck_list = []

    for key in cards.keys():
        for i in range(0,cards[key][1]):
            deck_list.append(cards[key][0])
    with open('deck.pkl', 'wb') as f:
        pickle.dump(deck_list, f)

In [ ]:
search_terms = {'quality': {'targetted discard', 
                            'win condition',
                            'recurring discard'}}

In [1]:
hand = deck[:7]

NameError: name 'deck' is not defined

In [ ]:
stats = shuffle(deck_list, search_terms, overhand_interval=3)

In [ ]:
number_of_shuffles = [stat['shuffles'] for stat in stats]

In [ ]:
print(f'Modal number of shuffles is: {max(set(number_of_shuffles), key=number_of_shuffles.count)}')
print(f'Average number of shuffles is: {sum(number_of_shuffles)/len(number_of_shuffles)}')

dist = {x:number_of_shuffles.count(x) for x in number_of_shuffles}

scatter_keys = []
scatter_values = []

for key, value in dist.items():
    scatter_keys.append(key)
    scatter_values.append(value)

scatter_data = pd.DataFrame({'shuffles': scatter_keys,
                             'frequency': scatter_values})
    

a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)

seaborn.distplot(number_of_shuffles, ax=ax)
seaborn.lmplot('shuffles', 'frequency', scatter_data, fit_reg=False, size=10)

In [ ]:
total = 0
cumulative = []

keys = dist.keys()
keys = sorted(keys)

for key in keys:
    total = total + int(dist[key])
    cumulative.append(total)

In [ ]:
for i, item in enumerate(cumulative):
    print(i, 100 * float(item)/float(total))